# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the file with cities' weather data into a data frame
vacation_csv_path = "output_data/cities.csv"
vacation_df = pd.read_csv(vacation_csv_path)
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Faanui,-16.48,-151.75,77.23,78,23,17.58,PF,1595458253
1,Husavik,66.04,-17.34,48.20,81,90,9.17,IS,1595458513
2,Rozivka,46.94,35.77,62.40,62,9,7.96,UA,1595458771
3,Busselton,-33.65,115.33,57.00,48,33,20.00,AU,1595458500
4,Bluff,-46.60,168.33,39.00,83,96,3.00,NZ,1595458500
...,...,...,...,...,...,...,...,...,...
536,Birjand,32.87,59.22,78.80,15,0,9.17,IR,1595458837
537,Coari,-4.08,-63.14,78.78,77,35,1.41,BR,1595458820
538,Mahajanga,-15.72,46.32,66.88,62,66,9.44,MG,1595458837
539,Lumeje,-11.55,20.78,55.20,43,0,5.93,AO,1595458838


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = vacation_df["Humidity"].astype(float)

In [4]:
# Assign a gmaps figure with a zoom level and proper centering of the world map
fig = gmaps.figure(center=(35.0,-5.0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,max_intensity=100)
                                
# Add heatmap layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the cities to fit weather conditions
# Drop any rows with null values (if applicable)
fit_vacation_df = vacation_df[(vacation_df["Max Temp"]> 70) & (vacation_df["Max Temp"]< 80) & (vacation_df["Wind Speed"]< 10) & (vacation_df["Cloudiness"] == 0)]
fit_vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
114,Gaza,31.50,34.47,77.00,64,0,2.10,PS,1595458781
178,Abu Samrah,35.30,37.18,72.10,57,0,7.27,SY,1595458787
183,Kiryat Gat,31.61,34.76,77.00,63,0,1.72,IL,1595458527
201,Sabaudia,41.30,13.03,75.99,57,0,9.80,IT,1595458789
264,Arrondissement de Bastia,42.67,9.33,77.00,64,0,8.05,FR,1595458795
265,Belmonte,-15.86,-38.88,73.40,60,0,4.70,BR,1595458670
270,São Miguel do Araguaia,-13.28,-50.16,76.15,37,0,4.29,BR,1595458596
275,Irauçuba,-3.75,-39.78,73.80,55,0,3.85,BR,1595458796
289,Battipaglia,40.61,14.99,75.99,53,0,2.46,IT,1595458797
295,Tiznit Province,29.58,-9.50,79.54,41,0,1.32,MA,1595458798


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Store into variable named hotel_df
hotel_df = fit_vacation_df.copy()

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
114,Gaza,31.50,34.47,77.00,64,0,2.10,PS,1595458781,
178,Abu Samrah,35.30,37.18,72.10,57,0,7.27,SY,1595458787,
183,Kiryat Gat,31.61,34.76,77.00,63,0,1.72,IL,1595458527,
201,Sabaudia,41.30,13.03,75.99,57,0,9.80,IT,1595458789,
264,Arrondissement de Bastia,42.67,9.33,77.00,64,0,8.05,FR,1595458795,
265,Belmonte,-15.86,-38.88,73.40,60,0,4.70,BR,1595458670,
270,São Miguel do Araguaia,-13.28,-50.16,76.15,37,0,4.29,BR,1595458596,
275,Irauçuba,-3.75,-39.78,73.80,55,0,3.85,BR,1595458796,
289,Battipaglia,40.61,14.99,75.99,53,0,2.46,IT,1595458797,
295,Tiznit Province,29.58,-9.50,79.54,41,0,1.32,MA,1595458798,


In [9]:
# Iterate through the rows of the data frame to build an API query and get a response for each city with its geo coordinates

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

# Use the lat/lng we recovered to identify airports
for ind in hotel_df.index:
    target_coordinates = f"{hotel_df['Lat'][ind]},{hotel_df['Lng'][ind]}"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # Confirm that the response status returned is OK
    if response.status_code == 200:
        # convert response to json
        hotel_data = response.json()
        
        # Save the Hotel names returned into the data frame column 
        try:
            hotel_df.loc[ind, "Hotel Name"] = hotel_data["results"][0]["name"]

        except (ValueError, KeyError, IndexError):
            print("Missing field/result... skipping.")

    
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
114,Gaza,31.50,34.47,77.00,64,0,2.10,PS,1595458781,فندق جراند بالاس Grand Palace Hotel
178,Abu Samrah,35.30,37.18,72.10,57,0,7.27,SY,1595458787,
183,Kiryat Gat,31.61,34.76,77.00,63,0,1.72,IL,1595458527,Desert Gat
201,Sabaudia,41.30,13.03,75.99,57,0,9.80,IT,1595458789,Hotel le Dune
264,Arrondissement de Bastia,42.67,9.33,77.00,64,0,8.05,FR,1595458795,Hôtel Dolce Notte
265,Belmonte,-15.86,-38.88,73.40,60,0,4.70,BR,1595458670,Pousada Caçari
270,São Miguel do Araguaia,-13.28,-50.16,76.15,37,0,4.29,BR,1595458596,Hotel Veredas do Araguaia
275,Irauçuba,-3.75,-39.78,73.80,55,0,3.85,BR,1595458796,Pousada Blue Marlin
289,Battipaglia,40.61,14.99,75.99,53,0,2.46,IT,1595458797,Hotel San Luca - Centro Congressi
295,Tiznit Province,29.58,-9.50,79.54,41,0,1.32,MA,1595458798,


In [10]:
# Drop any NaN values
hotel_df = hotel_df.dropna()

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))